In [ ]:
# !pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git

In [10]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
from datetime import datetime
import time

In [11]:
# nifty_50_componants_data_from_tradingview.csv
nifty_comprehensive_df = pd.read_csv("D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv")
nifty_50 = nifty_comprehensive_df['Symbol'].values.tolist()
nifty_50

['RELIANCE',
 'HDFCBANK',
 'TCS',
 'BHARTIARTL',
 'ICICIBANK',
 'SBIN',
 'INFY',
 'BAJFINANCE',
 'HINDUNILVR',
 'ITC',
 'LT',
 'MARUTI',
 'M&M',
 'HCLTECH',
 'KOTAKBANK',
 'SUNPHARMA',
 'ULTRACEMCO',
 'AXISBANK',
 'BAJAJFINSV',
 'NTPC',
 'TITAN',
 'ETERNAL',
 'ADANIPORTS',
 'ONGC',
 'BEL',
 'ADANIENT',
 'JSWSTEEL',
 'POWERGRID',
 'WIPRO',
 'TATAMOTORS',
 'BAJAJ_AUTO',
 'ASIANPAINT',
 'COALINDIA',
 'NESTLEIND',
 'TATASTEEL',
 'JIOFIN',
 'GRASIM',
 'EICHERMOT',
 'SBILIFE',
 'TRENT',
 'HINDALCO',
 'HDFCLIFE',
 'TECHM',
 'CIPLA',
 'SHRIRAMFIN',
 'APOLLOHOSP',
 'DRREDDY',
 'TATACONSUM',
 'HEROMOTOCO',
 'INDUSINDBK']

In [12]:

# TradingView credentials
username = "truthfulOwl34362"
password = "asdfasdf12#$jkl;"

tv = TvDatafeed(username=username, password=password)

# Symbol: NIFTY_50, Exchange: NSE, Interval: 1 day, Bars: 500
nifty_data = tv.get_hist(
    symbol='NIFTY_50',
    exchange='NSE', 
    interval=Interval.in_daily,
    n_bars=1172
)

# Reset index and format data
nifty_data.reset_index(inplace=True)
nifty_data = nifty_data[['datetime', 'open', 'high', 'low', 'close', 'volume']]
nifty_data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
nifty_data['Date'] = nifty_data['Date'].dt.strftime('%Y-%m-%d')

# Round price values
for col in ['Open', 'High', 'Low', 'Close']:
    nifty_data[col] = nifty_data[col].round(2)

# Save to CSV
nifty_data.to_csv('nifty50_tradingview_data.csv', index=False)

nifty_data


ERROR:tvDatafeed.main:error while signin


,Date,Open,High,Low,Close,Volume
0,2021-01-01,13996.10,14049.85,13991.35,14018.50,258090910.0
1,2021-01-04,14104.35,14147.95,13953.75,14132.90,494999306.0
2,2021-01-05,14075.15,14215.60,14048.15,14199.50,492475351.0
3,2021-01-06,14240.95,14244.15,14039.90,14146.25,632323310.0
4,2021-01-07,14253.75,14256.25,14123.10,14137.35,559173534.0
...,...,...,...,...,...,...
1167,2025-09-16,25073.60,25261.40,25070.45,25239.10,240122320.0
1168,2025-09-17,25276.60,25346.50,25275.35,25330.25,268936695.0
1169,2025-09-18,25441.05,25448.95,25329.75,25423.60,272193683.0
1170,2025-09-19,25410.20,25428.75,25286.30,25327.05,380362832.0
